# Исследование надёжности заёмщиков

## Этап 1. Получение данных

Импорт библиотек.

In [1]:
import pandas as pd

Откроем файл `data.csv`, сохраним его в переменной `df` и посмотрим первые 10 строк таблицы.

In [2]:
df = pd.read_csv('data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Посмотрим общую информацию о данных таблицы `df`.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Всего в таблице 12 столбцов.

Разберём, какие в `df` столбцы и какую информацию они содержат:
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

### Выводы

Первое представление о данных получено. Можно сделать следующие выводы:

- Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения.
- В столбце `days_employed` наблюдаются аномалии - отрицательное количество дней трудового стажа, необходимо избавиться от отрицательных значений.
- В столбце `education` содержатся одинаковые записи с разным регистром, необходимо привести данные к единообразию.
- В столбце `purpose` присутствуют данные с близкими по смыслу значениями, необходимо привести данные к единообразию.

## Этап 2. Предобработка данных

### Обработка пропусков

Посмотрим, сколько пропусков в каждом столбце.

In [4]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски имеются в двух столбцах: `days_employed` и `total_income`.

Обработаем аномальные значения в столбце `days_employed`. Заменим все отрицательные значения положительными.

In [5]:
df['days_employed'] = df['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [6]:
df.groupby('income_type')['days_employed'].median().sort_values(ascending=False)

income_type
безработный        366413.652744
пенсионер          365213.306266
в декрете            3296.759962
госслужащий          2689.368353
сотрудник            1574.202821
компаньон            1547.382223
студент               578.751554
предприниматель       520.848083
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) аномально большие значения. Исправить такие значения сложно, поэтому оставим их в таком виде.

Заполним пропуски в столбце `days_employed` медианными значениями для каждого типа занятости `income_type`.

In [7]:
for t in df['income_type'].unique():
    df.loc[(df['income_type'] == t) & (df['days_employed'].isna()), 'days_employed'] = \
    df.loc[(df['income_type'] == t), 'days_employed'].median()

Столбец `total_income` хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца `income_type`.

In [8]:
for t in df['income_type'].unique():
    df.loc[(df['income_type'] == t) & (df['total_income'].isna()), 'total_income'] = \
    df.loc[(df['income_type'] == t), 'total_income'].median()

Убедимся, что в таблице не осталось пропусков.

In [9]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Обработка аномальных значений

Выведем перечень уникальных значений столбца `children` и посмотрим на распределение данных по категориям. 

In [10]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В данных присутствуют аномальные значения, например, 20 и -1. Предположим, что в данных ошибка и выполним следующие действия: 
- заменим `20` на `2`. 
- заменим `-1` на `1`.

In [11]:
df['children'] = df['children'].replace(20, 2)
df['children'] = df['children'].replace(-1, 1)

Убедимся, что аномальные значения в столбце `children` заменены. 

In [12]:
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Выведем перечень уникальных значений столбца `dob_years`.

In [13]:
df['dob_years'].sort_values().unique() 

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

Вряд ли возраст клиента может составлять `0` лет. Заменим аномальные значения в столбце на медиану.

In [14]:
df['dob_years'] = df['dob_years'].replace(0, df['dob_years'].median())

Проверим, что аномалии действительно удалены.

In [15]:
df['dob_years'].sort_values().unique() 

array([19., 20., 21., 22., 23., 24., 25., 26., 27., 28., 29., 30., 31.,
       32., 33., 34., 35., 36., 37., 38., 39., 40., 41., 42., 43., 44.,
       45., 46., 47., 48., 49., 50., 51., 52., 53., 54., 55., 56., 57.,
       58., 59., 60., 61., 62., 63., 64., 65., 66., 67., 68., 69., 70.,
       71., 72., 73., 74., 75.])

Выведем перечень уникальных значений столбца `education_id`.

In [16]:
df['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

Аномалий не обнаружено.

Выведем перечень уникальных значений столбца `family_status`.

In [17]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Аномалий не обнаружено.

Выведем перечень уникальных значений столбца `family_status_id`.

In [18]:
df['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

Аномалий не обнаружено.

Выведем перечень уникальных значений столбца `gender`.

In [19]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Удалим запись с данными `XNA`.

In [20]:
df = df[df['gender'] != 'XNA']

Выведем перечень уникальных значений столбца `income_type`.

In [21]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5084
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

Аномалий не обнаружено.

Выведем перечень уникальных значений столбца `debt`.

In [22]:
df['debt'].value_counts()

0    19783
1     1741
Name: debt, dtype: int64

Аномалий не обнаружено.

Выведем перечень уникальных значений столбца `purpose`.

In [23]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      623
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Аномалий не обнаружено.

### Обработка дубликатов

Выведем перечень уникальных значений столбца `education`.

In [24]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      667
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Приведем все данные в столбце к нижнему регистру.

In [25]:
df['education'] = df['education'].str.lower()

Проверим, что теперь данные в столбце единообразны.

In [26]:
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

Посмотрим, есть ли явные дубликаты в таблице.

In [27]:
df.duplicated().sum()

72

Удалим дубликаты.

In [28]:
df = df.drop_duplicates().reset_index(drop=True)

Убедимся, что все дубликаты удалены.

In [29]:
df.duplicated().sum()

0

### Изменение типов данных

Еще раз посмотрим на общую информацию о данных.

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21452 non-null  int64  
 1   days_employed     21452 non-null  float64
 2   dob_years         21452 non-null  float64
 3   education         21452 non-null  object 
 4   education_id      21452 non-null  int64  
 5   family_status     21452 non-null  object 
 6   family_status_id  21452 non-null  int64  
 7   gender            21452 non-null  object 
 8   income_type       21452 non-null  object 
 9   debt              21452 non-null  int64  
 10  total_income      21452 non-null  float64
 11  purpose           21452 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


Заменим тип данных `float` на `int`.

In [31]:
df[['days_employed', 'dob_years', 'total_income']] = \
    df[['days_employed', 'dob_years', 'total_income']].astype(int)

### Категоризация данных

Создадим столбец `total_income_category` со следующими категориями:
- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

Для этого напишем функцию:

In [32]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [33]:
df['total_income_category'] = df['total_income'].apply(categorize_income)

Напишем функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:
- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [34]:
def categorize_purpose(row):
    try:
        if 'авто' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образ' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [35]:
df['purpose_category'] = df['purpose'].apply(categorize_purpose)

Проверим, корректно ли добавились столбцы.

In [36]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


### Выводы

На этапе предобработки данных была проделана следующая работа:
- заполнены пропуски в столбцах `days_employed` и `total_income` на медианные значения по каждому типу из столбца `income_type`,
- заменены аномальные значения в столбцах `days_employed`, `children`, `dob_years`, 
- удалено аномальное значение в столбце `gender`,
- удалены явные дубликаты в таблице,
- данные в столбце `education` приведены к нижнему регистру, что позволило избавиться от неявных дубликатов,
- в столбцах `days_employed`, `dob_years`, `total_income` тип данных `float` заменен на `int`,
- в таблицу добавлены следующие столбцы:
  - `total_income_category` с категориям дохода,
  - `purpose_category` с категориями цели кредита.

## Этап 3. Проверка гипотез

### Существует зависимость между количеством детей и возвратом кредита в срок.

In [37]:
df_pivot = df.pivot_table(index=['children'], columns='debt', values='family_status_id', aggfunc='count') 
df_pivot.columns = ['No debt', 'Debt']
df_pivot['Total'] = df_pivot['No debt'] + df_pivot['Debt'] 
df_pivot['Not returned on time, %'] = df_pivot['Debt'] / df_pivot['Total']
print(df_pivot.sort_values(by='Not returned on time, %', ascending = False)) 

          No debt    Debt    Total  Not returned on time, %
children                                                   
4            37.0     4.0     41.0                 0.097561
2          1926.0   202.0   2128.0                 0.094925
1          4410.0   445.0   4855.0                 0.091658
3           303.0    27.0    330.0                 0.081818
0         13026.0  1063.0  14089.0                 0.075449
5             9.0     NaN      NaN                      NaN


**Вывод:** Исходя из полученных данных, мы видим, что наименьшая доля просрочек по кредиту у семей, в которых дети отсутствуют. Наибольшая доля просрочек по кредиту - в семьях с 4 детьми. В семьях с 5 детьми задолженности отсутствуют, но в предоставленных данных всего 9 подобных семей, делать выводы из такого малого количества данных не представляется возможным.

В целом, появление ребенка увеличивает возможность задолженности по кредиту 

### Существует зависимость между семейным положением и возвратом кредита в срок.

In [38]:
df_pivot = df.pivot_table(index=['family_status'], columns='debt', values='family_status_id', aggfunc='count') 
df_pivot.columns = ['No debt', 'Debt']
df_pivot['Total'] = df_pivot['No debt'] + df_pivot['Debt'] 
df_pivot['Not returned on time, %'] = df_pivot['Debt'] / df_pivot['Total'] 
print(df_pivot.sort_values(by='Not returned on time, %', ascending = False)) 

                       No debt  Debt  Total  Not returned on time, %
family_status                                                       
Не женат / не замужем     2536   274   2810                 0.097509
гражданский брак          3761   388   4149                 0.093517
женат / замужем          11408   931  12339                 0.075452
в разводе                 1110    85   1195                 0.071130
вдовец / вдова             896    63    959                 0.065693


**Вывод:** Наименьшая доля просрочек по кредиту у категории 'вдовец / вдова'. Наибольшая доля просрочек по кредиту - у категории 'Не женат / не замужем', чуть ниже - у тех, кто находится в гражданском браке.

Кредитному отделу следует с осторожностью относиться к клиентам, которые не состоят или не состояли в браке. Согласно данным такие клиенты менее ответственные. Можно сделать вывод, что узаконенные отношения ведут к более ответственному поведению и выполнению своих обязательств.

### Существует зависимость между уровнем дохода и возвратом кредита в срок.

In [39]:
    """
    0–30000 — 'E';
    30001–50000 — 'D';
    50001–200000 — 'C';
    200001–1000000 — 'B';
    1000001 и выше — 'A'.
    """

df_pivot = df.pivot_table(index=['total_income_category'], columns='debt', values='family_status_id', aggfunc='count') 
df_pivot.columns = ['No debt', 'Debt']
df_pivot['Total'] = df_pivot['No debt'] + df_pivot['Debt'] 
df_pivot['Not returned on time, %'] = df_pivot['Debt'] / df_pivot['Total'] 
print(df_pivot.sort_values(by='Not returned on time, %', ascending = False)) 

                       No debt  Debt  Total  Not returned on time, %
total_income_category                                               
E                           20     2     22                 0.090909
C                        14654  1360  16014                 0.084926
A                           23     2     25                 0.080000
B                         4685   356   5041                 0.070621
D                          329    21    350                 0.060000


**Вывод:** Наибольшая доля просрочек по кредиту у людей с уровнем дохода E, то есть наименьшим уровнем (до 30000 рублей). Наименьшая доля просрочек по кредиту у людей с уровнем дохода от 30001 до 50000 рублей. 

Выборка по заемщикам, как с наименьшим доходом, так и с наибольшим, крайне недостаточна для формирования однозначных выводов. Можно думать, что люди, с уровнем дохода E не могут корректно распоряжаться деньгами и «влезают» в кредиты, но тогда не совсем понятно, почему у людей с уровнем дохода А также достаточно высокий процент задолженности.

Если рассматривать для анализа только группы В, С и D, то по полученным данным не наблюдаем зависимости "те, кто получает больший доход, лучше возвращает кредит». В целом, материальный достаток - не единственная переменная, которая влияет на качество кредитной истории.

### Разные цели кредита влияют на его возврат в срок.

In [40]:
df_pivot = df.pivot_table(index=['purpose_category'], columns='debt', values='family_status_id', aggfunc='count') 
df_pivot.columns = ['No debt', 'Debt']
df_pivot['Total'] = df_pivot['No debt'] + df_pivot['Debt'] 
df_pivot['Not returned on time, %'] = df_pivot['Debt'] / df_pivot['Total'] 
print(df_pivot.sort_values(by='Not returned on time, %', ascending = False)) 

                          No debt  Debt  Total  Not returned on time, %
purpose_category                                                       
операции с автомобилем       3903   403   4306                 0.093590
получение образования        3643   370   4013                 0.092200
проведение свадьбы           2137   186   2323                 0.080069
операции с недвижимостью    10028   782  10810                 0.072340


**Вывод:** Наибольшая доля просрочек у людей, которые берут кредит для операций с автомобилем и для получения образования. Часто люди покупают в кредит автомобили гораздо дороже, чем те, что соответствуют их доходам.

Наименьшая доля просрочек у людей, которые берут кредит для операций с недвижимостью, люди стараются не рисковать своим жильем.

## Итоги исследования

Было проведено исследование для выявления влияния различных факторов на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Перед проведением исследования были поставлены гипотезы:
- Гипотеза 1. Существует зависимость между количеством детей и возвратом кредита в срок.
- Гипотеза 2. Существует зависимость между семейным положением и возвратом кредита в срок.
- Гипотеза 3. Существует зависимость между уровнем дохода и возвратом кредита в срок.
- Гипотеза 4. Разные цели кредита влияют на его возврат в срок.

В результате проведенного исследования удалось выявить следующие зависимости:

**Гипотеза 1 подтверждена частично**
- 7,5% - доля просроченных кредитов среди заемщиков без детей 
- 9,2% - доля просроченных кредитов среди заемщиков с 1 ребенком
- 9,8% - доля просроченных кредитов среди заемщиков с 4 детьми

Высокой разницы между группами заемщиков нет, тем не менее, можно сделать вывод о том, что заемщики без детей являются более надежными и ответственными, чем заемщики с детьми.

Рекомендации: для более точного анализа необходима выборка с бОльшим объемом данных.
 

**Гипотеза 2 подтверждена**
- Заемщики, не состоящие никогда в браке (9,8%) и проживающие в гражданском браке (9,3%) являются ненадежными, к таким заемщикам следует относиться с осторожностью.
- Наиболее ответственными заемщиками являются вдовцы/вдовы (6,6%), состоящие в разводе (7%), а также состоящие в браке (7,5%)
*в скобках приведена доля просроченных кредитов.


**Гипотеза 3 подтверждена частично**
- Наиболее ответственными заемщиками являются люди с уровнем дохода от 30001 до 50000 рублей (6%). 
- Заемщики с уровнем дохода до 30000 рублей (9%) являются наименее надежными.
*в скобках приведена доля просроченных кредитов.

Как группа людей с наименьшим доходом, так и группа с наибольшим доходом содержат слишком мало записей. Вероятно, это связано с большим количеством пропущенным значений по столбцу с ежемесячным доходом, что искажает полученные результаты.

Рекомендации: для более точного анализа необходима выборка с бОльшим объемом данных. Также стоит учитывать сумму кредита, а также регион проживания заемщика, ведь доходы довольно сильно различаются по регионам.


**Гипотеза 4 подтверждена**
- 9,3% - доля просроченных кредитов среди заемщиков, целью которых являются операции с автомобилем.
- 9,2% - доля просроченных кредитов среди заемщиков, целью которых является получение образования.
- 7,9% - доля просроченных кредитов среди заемщиков, целью которых является проведение свадьбы.
- 7,2% - доля просроченных кредитов среди заемщиков, целью которых являются операции с недвижимостью.

Заемщики, решающие приобрести недвижимость, являются более ответственными, так как в случае просрочек и невыплат по кредиту есть риск остаться без жилья.


**Общие рекомендации:** 
1. Специалистам банка можно использовать для работы подтвержденные гипотезы:
    - Гипотеза 2. Существует зависимость между семейным положением и возвратом кредита в срок;
    - Гипотеза 4. Разные цели кредита влияют на его возврат в срок.
2. Для более точного анализа необходима выборка с бОльшим объемом данных, а также добавление других факторов, которые могут влиять на факт погашения кредита в срок.